In [13]:
from pymongo import MongoClient
from pprint import pprint as pp

In [14]:
# CONNECT TO DATABASE

client = MongoClient('localhost', 27017)
db = client['project']

In [15]:
# SHOW COLLECTIONS

for collection in db.list_collection_names():
    print(collection)

internetData
solarData


In [16]:
db.internetData.find_one()

{'_id': ObjectId('62cb0b1d23802562df01925d'),
 'quadkey': 1233001311211132,
 'avg_d_kbps': 1366,
 'avg_u_kbps': 2670,
 'avg_lat_ms': 148,
 'tests': 2,
 'devices': 1,
 'year': '2019',
 'quarter': 'Q4'}

In [29]:
db.solarData.find_one()

{'_id': ObjectId('62cb3516d0eb9a592bc06b4e'),
 '#YYY': 1932.0,
 'MM': 1.0,
 'DD': 1.0,
 'hh.h': 0.0,
 'hh._m': 1.5,
 'days': 0.0,
 'days_m': 0.0625,
 'Kp': 3.333,
 'ap': 18.0,
 'D': 1.0}

In [34]:
def choose_year_quarter():
    
    year1 = input('PLEASE CHOOSE 1ST YEAR')
    mapquarter1  = input('PLEASE CHOOSE 1ST QUARTER')
    year2 = input('PLEASE CHOOSE 2ND YEAR')
    mapquarter2  = input('PLEASE CHOOSE 2ND QUARTER')

    mapyear = {
        year1: 51400, # 51080
        year2: 84000
    }

    mapquarter  = {year1: mapquarter1, year2 : mapquarter2}

    lesserCounts_test_1 = []


    for year, mean in mapyear.items():
        lesserCounts_test_1.append(db.internetData.aggregate([ #calcul du nombre de test ayant une avg_d_kbps inférieur au seuil de la moyenne.
            {"$match": { "year":year, "quarter": mapquarter[year], "avg_d_kbps": {"$lt": mean}}},
            {"$group": {"_id": "$year", "count_kbps": {"$sum": 1}}}
        ]))
        lesserCounts_test_1.append(db.internetData.aggregate([ #calcul du nombre de test ayant une avg_lat_ms supérieur à la moyenne
            {"$match": { "year":year, "quarter": mapquarter[year], "avg_lat_ms": {"$gt": 84}}},
            {"$group": {"_id": "$year", "count_lat": {"$sum": 1}}}
        ]))
        
    for year in lesserCounts_test_1:
        for line in year:
            print(line)

In [35]:
choose_year_quarter()

{'_id': '2019', 'count_kbps': 4053747}
{'_id': '2019', 'count_lat': 314063}


In [ ]:
{'_id': '2019', 'count_kbps': 4053747}
{'_id': '2019', 'count_lat': 314063}
{'_id': '2021', 'count_kbps': 3995067}
{'_id': '2021', 'count_lat': 208964}

### VERIFIER L'HYPOTHESE

In [10]:
mapyear = {
    "2019": 51400, # 51080
    "2021": 84000
}

In [11]:
# TEST 1

lesserCounts_test_1 = []
mapquarter  = {"2019": "Q4", "2021":"Q4"}

for year, mean in mapyear.items():
    lesserCounts_test_1.append(db.internetData.aggregate([ #calcul du nombre de test ayant une avg_d_kbps inférieur au seuil de la moyenne.
        {"$match": { "year":year, "quarter": mapquarter[year], "avg_d_kbps": {"$lt": mean}}},
        {"$group": {"_id": "$year", "count_kbps": {"$sum": 1}}}
    ]))
    lesserCounts_test_1.append(db.internetData.aggregate([ #calcul du nombre de test ayant une avg_lat_ms supérieur à la moyenne
        {"$match": { "year":year, "quarter": mapquarter[year], "avg_lat_ms": {"$gt": 84}}},
        {"$group": {"_id": "$year", "count_lat": {"$sum": 1}}}
    ]))

In [8]:
# TEST 2

lesserCounts_test_2 = []
mapquarter  = {"2019": "Q2", "2021":"Q1"}

for year, mean in mapyear.items():
    lesserCounts_test_2.append(db.internetData.aggregate([ #calcul du nombre de test ayant une avg_d_kbps inférieur au seuil de la moyenne.
        {"$match": { "year":year, "quarter": mapquarter[year], "avg_d_kbps": {"$lt": mean}}},
        {"$group": {"_id": "$year", "count_kbps": {"$sum": 1}}}
    ]))
    lesserCounts_test_2.append(db.internetData.aggregate([ #calcul du nombre de test ayant une avg_lat_ms supérieur à la moyenne
        {"$match": { "year":year, "quarter": mapquarter[year], "avg_lat_ms": {"$gt": 84}}},
        {"$group": {"_id": "$year", "count_lat": {"$sum": 1}}}
    ]))

In [21]:
for year in lesserCounts_test_1:
    for line in year:
        print(line)

{'_id': '2019', 'count_kbps': 4053747}
{'_id': '2019', 'count_lat': 314063}
{'_id': '2021', 'count_kbps': 3995067}
{'_id': '2021', 'count_lat': 208964}


In [7]:
for year in lesserCounts_test_1:
    for line in year:
        print(line)

{'_id': '2019', 'kbps': 4053747}
{'_id': '2019', 'lat': 314063}
{'_id': '2021', 'kbps': 3995067}
{'_id': '2021', 'lat': 208964}


count nb de kbps < moyenne

In [9]:
for year in lesserCounts_test_2:
    for line in year:
        print(line)

{'_id': '2019', 'kbps': 3202613}
{'_id': '2019', 'lat': 321742}
{'_id': '2021', 'kbps': 5023107}
{'_id': '2021', 'lat': 251263}


In [ ]:
{'_id': '2019Q4', 'kpStdDev': 1.014370301957935, 'maxKP': 5.333}
{'_id': '2021Q4', 'kpStdDev': 1.2169523948941285, 'maxKP': 7.667}


# 2019 -> on prend Q2 avec Kp = 6.333  
# 2021 -> on prend Q1 avec Kp = 5.667

In [8]:
years = ['2019', '2021']
mean_ = []

for year in years:
    mean_.append(db.internetData.aggregate([
                {"$match": { "year": year}},
                {"$group": {"_id": "$year", "count": {"$avg": "$avg_d_kbps"}}}
            ]))

In [9]:
for line in mean_:
    for l in line:
        print(l)

{'_id': '2019', 'count': 55941.952499305815}
{'_id': '2021', 'count': 90887.57912566785}


In [11]:
et = []

for y in range(2018,2022):
    for q in range(1,5):
        et.append(db.solarData.aggregate([
            {"$match": {"#YYY": y, "MM": {"$gte": q*3-2, "$lte": q*3}}},
            {"$group": {"_id": str(y)+"Q"+str(q), "kpStdDev": {"$stdDevSamp": "$Kp"}, "maxKP": {"$max": "$Kp"}}}
        ]))

cpt=0
for year in et:
    if cpt%4 == 0:
        print(" ")
    for line in year:
        print(line)
    cpt+=1

 
{'_id': '2018Q1', 'kpStdDev': 1.1138559006376263, 'maxKP': 6.0}
{'_id': '2018Q2', 'kpStdDev': 1.130007554356315, 'maxKP': 6.0}
{'_id': '2018Q3', 'kpStdDev': 1.1283575146673, 'maxKP': 7.333}
{'_id': '2018Q4', 'kpStdDev': 1.1196041741363634, 'maxKP': 5.667}
 
{'_id': '2019Q1', 'kpStdDev': 1.107465812686389, 'maxKP': 5.333}
{'_id': '2019Q2', 'kpStdDev': 0.9531569363219395, 'maxKP': 6.333}
{'_id': '2019Q3', 'kpStdDev': 1.1097021781372693, 'maxKP': 5.667}
{'_id': '2019Q4', 'kpStdDev': 1.014370301957935, 'maxKP': 5.333}
 
{'_id': '2020Q1', 'kpStdDev': 0.9468942338712437, 'maxKP': 4.333}
{'_id': '2020Q2', 'kpStdDev': 0.7885328735037124, 'maxKP': 4.667}
{'_id': '2020Q3', 'kpStdDev': 1.0968653644796296, 'maxKP': 5.667}
{'_id': '2020Q4', 'kpStdDev': 1.0344010510712278, 'maxKP': 4.667}
 
{'_id': '2021Q1', 'kpStdDev': 1.2921783578386619, 'maxKP': 5.667}
{'_id': '2021Q2', 'kpStdDev': 1.110008277490653, 'maxKP': 7.0}
{'_id': '2021Q3', 'kpStdDev': 0.9568817691776172, 'maxKP': 5.333}
{'_id': '2021Q4

In [12]:
# 2019 -> on prend Q2 avec Kp = 6.333  
# 2021 -> on prend Q1 avec Kp = 5.667